In [20]:
import tkinter as tk

def play_encrypt(text1, text2, text4):
    text = text1.get("1.0", "end")
    key = text2.get("1.0", "end")
    
    def toLowerCase(text):
        return text.lower()

    def removeSpaces(text):
        newText = ""
        for i in text:
            if i == " ":
                continue
            else:
                newText = newText + i
        return newText

    def Diagraph(text):
        Diagraph = []
        group = 0
        for i in range(2, len(text), 2):
            Diagraph.append(text[group:i])
            group = i
        Diagraph.append(text[group:])
        return Diagraph

    def FillerLetter(text):
        k = len(text)
        if k % 2 == 0:
            for i in range(0, k, 2):
                if text[i] == text[i+1]:
                    new_word = text[0:i+1] + str('x') + text[i+1:]
                    new_word = FillerLetter(new_word)
                    break
                else:
                    new_word = text
        else:
            for i in range(0, k-1, 2):
                if text[i] == text[i+1]:
                    new_word = text[0:i+1] + str('x') + text[i+1:]
                    new_word = FillerLetter(new_word)
                    break
                else:
                    new_word = text
        return new_word

    list1 = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm',
             'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

    def generateKeyTable(word, list1):
        key_letters = []
        for i in word:
            if i not in key_letters:
                key_letters.append(i)

        compElements = []
        for i in key_letters:
            if i not in compElements:
                compElements.append(i)
        for i in list1:
            if i not in compElements:
                compElements.append(i)

        matrix = []
        while compElements:
            matrix.append(compElements[:5])
            compElements = compElements[5:]

        return matrix

    def search(mat, element):
        for i in range(5):
            for j in range(5):
                if mat[i][j] == element:
                    return i, j
        return -1, -1 

    def encrypt_RowRule(matr, e1r, e1c, e2r, e2c):
        char1 = ''
        if e1c == 4:
            char1 = matr[e1r][0]
        else:
            char1 = matr[e1r][e1c+1]

        char2 = ''
        if e2c == 4:
            char2 = matr[e2r][0]
        else:
            char2 = matr[e2r][e2c+1]

        return char1, char2

    def encrypt_ColumnRule(matr, e1r, e1c, e2r, e2c):
        char1 = ''
        if e1r == 4:
            char1 = matr[0][e1c]
        else:
            char1 = matr[e1r+1][e1c]

        char2 = ''
        if e2r == 4:
            char2 = matr[0][e2c]
        else:
            char2 = matr[e2r+1][e2c]

        return char1, char2

    def encrypt_RectangleRule(matr, e1r, e1c, e2r, e2c):
        char1 = matr[e1r][e2c]
        char2 = matr[e2r][e1c]
        return char1, char2

    def encryptByPlayfairCipher(Matrix, plainList):
        cipherText = ""
        for i in range(len(plainList)):
            ele1_x, ele1_y = search(Matrix, plainList[i][0])
            ele2_x, ele2_y = search(Matrix, plainList[i][1])

            if ele1_x != -1 and ele1_y != -1 and ele2_x != -1 and ele2_y != -1:
                if ele1_x == ele2_x:
                    c1, c2 = encrypt_RowRule(Matrix, ele1_x, ele1_y, ele2_x, ele2_y)
                elif ele1_y == ele2_y:
                    c1, c2 = encrypt_ColumnRule(Matrix, ele1_x, ele1_y, ele2_x, ele2_y)
                else:
                    c1, c2 = encrypt_RectangleRule(Matrix, ele1_x, ele1_y, ele2_x, ele2_y)
                cipherText += c1 + c2
            else:
                # Handle the case when an element pair is not found
                # For simplicity, you can append 'x' as a placeholder
                cipherText += 'x'

        return cipherText


    text = removeSpaces(toLowerCase(text))
    PlainTextList = Diagraph(FillerLetter(text))
    if len(PlainTextList[-1]) != 2:
        PlainTextList[-1] = PlainTextList[-1]+'z'

    key = toLowerCase(removeSpaces(key))
    Matrix = generateKeyTable(key, list1)

    cipherText = encryptByPlayfairCipher(Matrix, PlainTextList)
    text4.delete("1.0", tk.END)  # Clear the text box before inserting
    text4.insert("1.0", cipherText)
    

    
    
    
# Decryption handling correct
def decryptByPlayfairCipher(str, key):
    def toLowerCase(plain):
        return plain.lower()

    def removeSpaces(plain):
        return ''.join(plain.split())

    def generateKeyTable(key):
        keyT = [['' for i in range(5)] for j in range(5)]
        dicty = {chr(i + 97): 0 for i in range(26)}

        for i in range(len(key)):
            if key[i] != 'j':
                dicty[key[i]] = 2
        dicty['j'] = 1

        i, j, k = 0, 0, 0
        while k < len(key):
            if dicty[key[k]] == 2:
                dicty[key[k]] -= 1
                keyT[i][j] = key[k]
                j += 1
                if j == 5:
                    i += 1
                    j = 0
            k += 1

        for k in dicty.keys():
            if dicty[k] == 0:
                keyT[i][j] = k
                j += 1
                if j == 5:
                    i += 1
                    j = 0

        return keyT

    def search(keyT, a, b):
        arr = [0, 0, 0, 0]

        if a == 'j':
            a = 'i'
        elif b == 'j':
            b = 'i'

        for i in range(5):
            for j in range(5):
                if keyT[i][j] == a:
                    arr[0], arr[1] = i, j
                elif keyT[i][j] == b:
                    arr[2], arr[3] = i, j

        return arr

    def mod5(a):
        if a < 0:
            a += 5
        return a % 5

    def decrypt(str, keyT):
        ps = len(str)
        i = 0
        while i < ps:
            a = search(keyT, str[i], str[i+1])
            if a[0] == a[2]:
                str = str[:i] + keyT[a[0]][mod5(a[1]-1)] + keyT[a[0]][mod5(a[3]-1)] + str[i+2:]
            elif a[1] == a[3]:
                str = str[:i] + keyT[mod5(a[0]-1)][a[1]] + keyT[mod5(a[2]-1)][a[1]] + str[i+2:]
            else:
                str = str[:i] + keyT[a[0]][a[3]] + keyT[a[2]][a[1]] + str[i+2:]
            i += 2

        return str

    key = removeSpaces(toLowerCase(key))
    str = removeSpaces(toLowerCase(str))
    keyT = generateKeyTable(key)
    return decrypt(str, keyT)

def play_decrypt(text1, text2, text4):
    text = text1.get("1.0", "end")
    key = text2.get("1.0", "end")
    
    decrypted_text = decryptByPlayfairCipher(text, key)
    text4.delete("1.0", tk.END)  # Clear the text box before inserting
    text4.insert("1.0", decrypted_text)
    
    


def open_window2():
    window2 = tk.Toplevel(window)
    window2.geometry("500x500")
    window2.title("Playfair Cipher Program")
    window2.configure(bg="bisque")

    label = tk.Label(window2, text="Playfair Cipher")
    label.config(font=("Arial", 30))
    label.pack(padx=20, pady=20)

    label2 = tk.Label(window2, text="Enter text for encryption or decryption", fg="black", font=("Arial", 13))
    label2.place(x=80, y=90)
    text1 = tk.Text(window2, font="Roboto 10", bg="white", bd=0)
    text1.place(x=65, y=120, width=355, height=100)

    label3 = tk.Label(window2, text="Enter the key", fg="black", font=("Arial", 13))
    label3.place(x=80, y=230)
    text2 = tk.Text(window2, font="Roboto 10", bg="white", bd=0)
    text2.place(x=65, y=260, width=355, height=20)

    text4 = tk.Text(window2, font="Roboto 10", bg="white", bd=0)
    text4.place(x=65, y=350, width=355, height=100)

    enc_button = tk.Button(window2, text="Encrypt", font=("Arial", 10), width=10, height=2, command=lambda: play_encrypt(text1, text2, text4))
    dec_button = tk.Button(window2, text="Decrypt", font=("Arial", 10), width=10, height=2,command=lambda: play_decrypt(text1, text2, text4))  # Add command for decryption
    enc_button.place(x=150, y=290)
    dec_button.place(x=250, y=290)

window = tk.Tk()
window.geometry("500x500")
window.title("Playfair Cipher Program")
window.configure(bg="bisque")

label = tk.Label(window, text="Playfair Cipher")
label.config(font=("Arial", 30))
label.pack(padx=10, pady=30)

enc_button = tk.Button(window, text="Start", font=("Arial", 15), width=15, height=2, command=open_window2)
enc_button.pack(padx=100, pady=100)

window.mainloop()
